Start by importing the functions

In [1]:
from pathlib import Path

from xnemogcm import open_domain_cfg, open_nemo, process_nemo, open_namelist, open_nemo_and_domain_cfg
from xnemogcm import __version__ as xnemogcm_version

In [2]:
xnemogcm_version

'0.4.1'

# First open the domain_cfg and nemo files into 2 datasets
## domain

In [3]:
open_domain_cfg?

---

You can provide the file names / folder using 3 similar methods:
1. Give the path to the files and xnemogcm opens the domain_cfg_out and mesh_mesk files
2. Give the path to the data folder + the name of the files
3. Give the name of the files that already contain the tree (e.g. ['/path/to/file1', '/path/to/file2']

These 3 methods are equivalent, however if your domain files don't have the strandard names you need to provide them by hand.

We use one of the test folder:

In [4]:
datadir = Path('../xnemogcm/test/data/4.2.0/open_and_merge/')

In [5]:
!ls ../xnemogcm/test/data/4.2.0/open_and_merge/

GYRE_1y_00010101_00011230_grid_T.nc  GYRE_1y_00010101_00011230_grid_W.nc
GYRE_1y_00010101_00011230_grid_U.nc  mesh_mask.nc
GYRE_1y_00010101_00011230_grid_V.nc


In [6]:
domcfg = open_domain_cfg(datadir=datadir)
# or
domcfg = open_domain_cfg(datadir=datadir, files=['mesh_mask.nc'])
# or
domcfg = open_domain_cfg(files=datadir.glob('*mesh_mask*.nc'))
domcfg

<xarray.Dataset>
Dimensions:    (z_c: 4, y_c: 22, x_c: 32, x_f: 32, y_f: 22, z_f: 4)
Coordinates: (12/18)
    glamt      (y_c, x_c) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    glamu      (y_c, x_f) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    glamv      (y_f, x_c) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    glamf      (y_f, x_f) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gphit      (y_c, x_c) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gphiu      (y_c, x_f) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    ...         ...
  * x_c        (x_c) int64 0 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30 31
  * y_c        (y_c) int64 0 1 2 3 4 5 6 7 8 9 ... 12 13 14 15 16 17 18 19 20 21
  * z_c        (z_c) int64 0 1 2 3
  * x_f        (x_f) float64 0.5 1.5 2.5 3.5 4.5 ... 27.5 28.5 29.5 30.5 31.5
  * y_f        (y_f) float64 0.5 1.5 2.5 3.5 4.5 ... 17.5 18.5 19.5 20.5 21.5
  * z_f        (z_f) float64 -0.5 0.5 1.5 2.5
Data variables: (12/28)
    tmask      (z_c, y_c, x_c) int8 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    umask      (z_c, y_c, x_f) int8 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    vmask      (z_c, y_f, x_c) int8 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    fmask      (z_c, y_f, x_f) int8 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    tmaskutil  (y_c, x_c) int8 dask.array<chunksize=(22, 32), meta=np.ndarray>
    umaskutil  (y_c, x_f) int8 dask.array<chunksize=(22, 32), meta=np.ndarray>
    ...         ...
    e3u_0      (z_c, y_c, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3v_0      (z_c, y_f, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3f_0      (z_c, y_f, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3w_0      (z_f, y_c, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3uw_0     (z_f, y_c, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3vw_0     (z_f, y_f, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
Attributes: (12/13)
    DOMAIN_dimensions_ids:   [1 2]
    DOMAIN_size_global:      [32 22]
    DOMAIN_halo_size_start:  [0 0]
    DOMAIN_halo_size_end:    [0 0]
    DOMAIN_type:             BOX
    CfgName:                 GYRE
    ...                      ...
    Iperio:                  0
    Jperio:                  0
    NFold:                   0
    NFtype:                  -
    VertCoord:               zco
    IsfCav:                  0

## Nemo

2 options here: 1) open netcdf files and make the preprocess automatically with `open_nemo` or 2) open by hand the files (or retrieve them from anywhere, e.g. zarr on a remote) and process using `process_nemo`.

Note: `open_nemo` internally uses `process_nemo`.

### open_nemo

In [7]:
open_nemo?

---
We can provide the files folder / name following the same convention as for the `open_domain_cfg` function. We also need to provide the `domcfg` dataset so xnemogcm knows how to set the variables on the proper grid position. We can also provide extra kwargs to the underlying call to `xarray.open_mfdataset` function.

In [8]:
nemo = open_nemo(domcfg=domcfg, datadir=datadir)
# or
nemo = open_nemo(domcfg=domcfg, files=datadir.glob('*grid*.nc'))
# or, using attributes from dataset and not name
datadir2 = Path('../xnemogcm/test/data/4.2.0/nemo_no_grid_in_filename/')
nemo = open_nemo(
    domcfg=domcfg, files=[
        datadir2 / 'T.nc',
        datadir2 / 'U.nc',
        datadir2 / 'V.nc',
        datadir2 / 'W.nc'
    ]
)
nemo

<xarray.Dataset>
Dimensions:               (z_c: 4, axis_nbounds: 2, t: 1, y_c: 22, x_c: 32,
                           x_f: 32, y_f: 22, z_f: 4)
Coordinates:
  * z_c                   (z_c) int64 0 1 2 3
    time_centered         (t) object dask.array<chunksize=(1,), meta=np.ndarray>
  * t                     (t) object 0001-07-01 00:00:00
  * x_c                   (x_c) int64 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * y_c                   (y_c) int64 0 1 2 3 4 5 6 7 ... 15 16 17 18 19 20 21
    gdept_1d              (z_c) float64 dask.array<chunksize=(4,), meta=np.ndarray>
  * x_f                   (x_f) float64 0.5 1.5 2.5 3.5 ... 28.5 29.5 30.5 31.5
  * y_f                   (y_f) float64 0.5 1.5 2.5 3.5 ... 18.5 19.5 20.5 21.5
  * z_f                   (z_f) float64 -0.5 0.5 1.5 2.5
    gdepw_1d              (z_f) float64 dask.array<chunksize=(4,), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/15)
    deptht_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(4, 2), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    toce                  (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    soce                  (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    e3t                   (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    ...                    ...
    depthv_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(4, 2), meta=np.ndarray>
    voce                  (t, z_c, y_f, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    e3v                   (t, z_c, y_f, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    depthw_bounds         (z_f, axis_nbounds) float32 dask.array<chunksize=(4, 2), meta=np.ndarray>
    woce                  (t, z_f, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    e3w                   (t, z_f, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    timeStamp:    2023-Mar-28 10:42:16 GMT
    name:         NEMO dataset
    description:  Ocean grid variables, set on the proper positions
    title:        Ocean grid variables

### process_nemo

In [9]:
process_nemo?

In [10]:
import xarray as xr
datadir2 = Path('../xnemogcm/test/data/4.2.0/nemo_no_grid_in_filename/')
nemo = process_nemo(
    positions=[
        (xr.open_dataset(datadir2 / 'T.nc'), 'T'),
        (xr.open_dataset(datadir2 / 'U.nc'), 'U'),
        (xr.open_dataset(datadir2 / 'V.nc'), 'V'),
        (xr.open_dataset(datadir2 / 'W.nc'), 'W')
    ],
    domcfg=domcfg
)
# or, if the datasets contain the attribute 'description'
nemo = process_nemo(
    positions=[
        (xr.open_dataset(datadir2 / 'T.nc'), None),
        (xr.open_dataset(datadir2 / 'U.nc'), None),
        (xr.open_dataset(datadir2 / 'V.nc'), None),
        (xr.open_dataset(datadir2 / 'W.nc'), None)
    ],
    domcfg=domcfg
)

# Open both at once

It is possible to open the domain and nemo output at once in one unique dataset. What happens is that 2 datasets are created and then merged. Thus all option possible for the `open_nemo` and `open_domain_cfg` functions are still possible.

In [11]:
open_nemo_and_domain_cfg?

---
Again, mutiple equivalent arguments are possible to open the data

In [12]:
# the simplest for simple cases, provide the path
ds = open_nemo_and_domain_cfg(nemo_files=datadir, domcfg_files=datadir)
# or provide the files
ds = open_nemo_and_domain_cfg(nemo_files=datadir.glob('*grid*.nc'), domcfg_files=datadir.glob('*mesh*.nc'))
# or use the nemo_kwargs and domcfg_kwargs dictionnaries
ds = open_nemo_and_domain_cfg(nemo_kwargs=dict(datadir=datadir), domcfg_kwargs={'files':datadir.glob('*mesh*.nc')})
ds

<xarray.Dataset>
Dimensions:               (z_c: 4, axis_nbounds: 2, t: 1, y_c: 22, x_c: 32,
                           x_f: 32, y_f: 22, z_f: 4)
Coordinates: (12/20)
  * z_c                   (z_c) int64 0 1 2 3
    time_centered         (t) object dask.array<chunksize=(1,), meta=np.ndarray>
  * t                     (t) object 0001-07-01 00:00:00
  * x_c                   (x_c) int64 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * y_c                   (y_c) int64 0 1 2 3 4 5 6 7 ... 15 16 17 18 19 20 21
    gdept_1d              (z_c) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    ...                    ...
    gphit                 (y_c, x_c) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gphiu                 (y_c, x_f) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gphiv                 (y_f, x_c) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gphif                 (y_f, x_f) float64 dask.array<chunksize=(22, 32), meta=np.ndarray>
    gdept_0               (z_c, y_c, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    gdepw_0               (z_f, y_c, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/43)
    deptht_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(4, 2), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    toce                  (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    soce                  (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    e3t                   (t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 4, 22, 32), meta=np.ndarray>
    ...                    ...
    e3u_0                 (z_c, y_c, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3v_0                 (z_c, y_f, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3f_0                 (z_c, y_f, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3w_0                 (z_f, y_c, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3uw_0                (z_f, y_c, x_f) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
    e3vw_0                (z_f, y_f, x_c) float64 dask.array<chunksize=(4, 22, 32), meta=np.ndarray>
Attributes: (12/18)
    Conventions:             CF-1.6
    timeStamp:               2023-Mar-28 10:42:16 GMT
    name:                    NEMO dataset
    description:             Ocean grid variables, set on the proper positions
    title:                   Ocean grid variables
    DOMAIN_dimensions_ids:   [1 2]
    ...                      ...
    Iperio:                  0
    Jperio:                  0
    NFold:                   0
    NFtype:                  -
    VertCoord:               zco
    IsfCav:                  0

## Remark

All opening are lazy using dask, which makes files quick to open, until you actually load the data you need

# Namelist

It can be convenient to open the namelist used for the run (e.g. to compare different runs with different parameters). This is possible using the `f90nml` package (it needs to be installed, this is an optional dependency).

In [13]:
open_namelist?

---
Here you provide the folder path containing the reference and configuration namelists, or the filenames (as for nemo and domcfg). You can choose to load both, or only one of them. The configration namelist will overwrite the default one.

For this we need to use another folder of the test data (with simplified namelists for the exemple):

In [14]:
datadir = Path('../xnemogcm/test/data/namelist/')

In [15]:
!ls ../xnemogcm/test/data/namelist/

namelist_cfg  namelist_ref


In [16]:
nam = open_namelist(datadir)
nam

<xarray.Dataset>
Dimensions:    ()
Data variables:
    nn_no      int64 0
    cn_exp     <U5 'BASIN'
    nn_it000   int64 1
    nn_itend   int64 5840
    rn_e1_deg  float64 1.0

In [17]:
nam.nn_it000

<xarray.DataArray 'nn_it000' ()>
array(1)
Attributes:
    namelist:  namrun